# M5 Competition

## Introduction

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/temporian/blob/main/docs/src/tutorials/m5_competition.ipynb)

The M5 Competition, held in 2020, was part of the prestigious Makridakis Forecasting Competitions. The goal of this competition was to accurately forecast the sales of 3,000 individual items across 10 Walmart stores for the next 28 days. The winning strategy involved transforming and augmenting the raw multivariate time series data into a tabular dataset, which was then used to train a collection of Gradient Boosted Trees models.

See https://www.kaggle.com/competitions/m5-forecasting-accuracy/overview.

In this notebook, we show how Temporian can replicate such data preprocessing and feature engineering with low effort. More specifically, we will replicate, with much less code, the preprocessing demonstrated at the [Khipu 2023: A Hands-On Forecasting Guide](https://colab.sandbox.google.com/drive/1nJ7GP0kayoCt6J-QkhyJw8jvr5uLnLcy), which was done with Pandas.

The notebook is divided into three parts:

- Loading the dataset and initial cleaning using Pandas.
- Temporal feature engineering using Temporian.
- Training and evaluation of a model using TensorFlow Decision Forests.

The following preprocessing operations will be applied using Temporian:

- **label**: Compute forecasting labels i.e. future leak of sales data.
- **lagged sales**: Give access to previous sales data.
- **moving statistics**: Moving average, sum and standard deviation of the sales over the last 7, 14, 28 and 64 days.
- **calendar features**: Extract discriminative calendar features such as day of the week, day of the month, etc.
- **aggregated sales**: Aggregate sales per department. Each item has access to the sales of its corresponding department.
- **special events**: Holidays, sales, and other special events.
- **train / test split**: Split the data between a training and testing dataset.

## Install and import dependencies


In [ ]:
%pip install temporian -q

# The Bokeh package is used by Temporian to create interactive plots.
# If if Bokeh is not installed, Temporian plots are static.
%pip install bokeh -q

In [ ]:
import os
import urllib.request
import zipfile
from collections import defaultdict
from datetime import datetime, timedelta, timezone
from typing import List

import pandas as pd
import temporian as tp

## Download the raw data

The M5 dataset is a collection of csv files.


In [ ]:
# Directory to download the raw M5 dataset and to export the result.
work_directory = "tmp/temporian_m5"
os.makedirs(work_directory, exist_ok=True)

# Download the M5 dataset
raw_data_zip = os.path.join(work_directory, "raw.zip")
if not os.path.exists(raw_data_zip):
    url = "https://docs.google.com/uc?export=download&id=1NYHXmgrcXg50zR4CVWPPntHx9vvU5jbM&confirm=t"
    urllib.request.urlretrieve(url, raw_data_zip)

# Extract the M5 dataset
raw_data_dir = os.path.join(work_directory, "raw")
if not os.path.exists(raw_data_dir):
    with zipfile.ZipFile(raw_data_zip, "r") as zip_ref:
        zip_ref.extractall(raw_data_dir)

In [ ]:
!ls -lh tmp/temporian_m5/raw

## Load raw dataset

The dataset is a set of csv files. We are using Pandas to load it and do some initial non-temporal preparations.


In [ ]:
raw_path = lambda x: os.path.join(raw_data_dir, x)

raw_sales = pd.read_csv(raw_path("sales_train_evaluation.csv"))
raw_sell_prices = pd.read_csv(raw_path("sell_prices.csv"))
raw_calendar = pd.read_csv(raw_path("calendar.csv"))

# Print the name, type and memory usage of the raw data columns.
print("==========\nraw_sales\n")
raw_sales.info()
print("==========\nraw_sell_prices\n")
raw_sell_prices.info()
print("==========\nraw_calendar_raw\n")
raw_calendar.info()

### Downsample the dataset

During development, we restrict the dataset to a subset of items for fast iteration.
Once you are happy with the result, we can use the full dataset with `downsample_dataset=False`.

Each product will result in ~1900 examples in the final tabular dataset. For example, using
a sample of 100 products will train a model on 190.000 examples which is more than enough for a quick demonstration.


In [ ]:
downsample_dataset = True

if downsample_dataset:
    # The items with the most sales.
    selected_items_names = ['FOODS_3_090', 'FOODS_3_586', 'FOODS_3_252', 'FOODS_3_555',
                            'FOODS_3_587', 'FOODS_3_714', 'FOODS_3_694', 'FOODS_3_226',
                            #'FOODS_3_202', 'FOODS_3_120', 'FOODS_3_723', 'FOODS_3_635',
                            #'FOODS_3_808', 'FOODS_3_377', 'FOODS_3_541', 'FOODS_3_080',
                            #'FOODS_3_318', 'FOODS_2_360', 'FOODS_3_681', 'FOODS_3_234',
                           ]
    
    sampled_raw_sales = raw_sales[raw_sales["item_id"].isin(selected_items_names)]
    print("Number of selected items:", len(sampled_raw_sales))

    raw_sales = sampled_raw_sales

### Normalize dataset

Each of the three tables `sales_raw`, `sell_prices_raw` and `calendar_raw` contains different information: `sales_raw` contains the daily sales of each product in each store, `sell_prices_raw` contains selling prices, and `calendar_raw` contains calendar events such as holidays.

Each table represents temporal data in a different way. For example, timestamps are stored by columns in `sales_raw` and by rows in `sell_prices_raw` and `calendar_raw`. The three tables also use a different system to represent dates.

In Temporian, the data is always expressed as an **EventSet**. Let's normalize and convert the raw tables to EventSets. EventSets can be built manually with `tp.event_set()` or they can be imported from Pandas DataFrames with `tp.from_pandas()`. In this case, timestamps need to be organized by rows.


#### Normalize `raw_sales`

Before:


In [ ]:
raw_sales.head()

Each daily sales value is stored in a different column. Instead, we want
for each value (which in Temporian we refer to as an event) to be in a different row.

For example, the record:

```
id,item_id,dept_id,d_1,d_2,d_3,...,d_n
```

will be converted to the following records:

```
id,item_id,dept_id,day,d_1
id,item_id,dept_id,day,d_2
...
id,item_id,dept_id,day,d_n
```


In [ ]:
cleaned_sales = pd.melt(
    raw_sales,
    var_name="day",
    value_name="sales",
    id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
)
cleaned_sales["day"] = cleaned_sales["day"].apply(lambda x: int(x[2:]))

The timestamps are expressed in number of days since `29/1/2011`.
Let's convert them into python datetimes.


In [ ]:
origin_date = datetime(2011, 1, 29, tzinfo=timezone.utc)
cleaned_sales["timestamp"] = cleaned_sales["day"].apply(
    lambda x: (origin_date + timedelta(days=x - 1))
)

Finally, we can remove the fields we won't use.


In [ ]:
del cleaned_sales["id"]

After:


In [ ]:
cleaned_sales.head()

Let's now convert `cleaned_sales` to a Temporian EventSet.


In [ ]:
sales_evset = tp.from_pandas(
    cleaned_sales,
    indexes=["item_id", "dept_id", "cat_id", "store_id", "state_id"],
)

sales_evset

Let's plot the sales of the first two products.


In [ ]:
sales_evset.plot(max_num_plots=4, interactive=True, width_px=500)

#### Normalize `raw_calendar`

In `raw_calendar`, timestamps are expressed as ISO 8601 string e.g., `2011-01-29`. Let's convert them into python datetimes.

Before:


In [ ]:
raw_calendar.head()

In [ ]:
cleaned_calendar = raw_calendar.copy()

# Temporian and TensorFlow Decision Forests (after) treat NaN values as "missing".
# In this dataset, a NaN means that there is not calendar event on this day.
cleaned_calendar.fillna("no_event", inplace=True)

cleaned_calendar["timestamp"] = cleaned_calendar["date"].apply(
    lambda x: datetime.strptime(x, "%Y-%m-%d")
)

# We keep the mapping wm_yr_wk -> timestamp to clean "raw_sell_prices" in the next section.
wm_yr_wk_map = cleaned_calendar[["weekday", "timestamp", "wm_yr_wk"]]

del cleaned_calendar["date"]
del cleaned_calendar["wm_yr_wk"]
del cleaned_calendar["d"]
del cleaned_calendar["weekday"]
del cleaned_calendar["wday"]
del cleaned_calendar["month"]
del cleaned_calendar["year"]

After:


In [ ]:
cleaned_calendar.head()

Same as before, we can convert the calendar data into a Temporian EventSet.


In [ ]:
calendar_evset = tp.from_pandas(cleaned_calendar)

calendar_evset

#### Normalize `raw_sell_prices`

Before:


In [ ]:
raw_sell_prices.head()

In `sell_prices`, timestamps as expressed with a special date format called `wm_yr_wk`.
We use the `calendar` data to find the mapping between `wm_yr_wk` and classical python datetimes.


In [ ]:
wm_yr_wk_to_date = wm_yr_wk_map[wm_yr_wk_map["weekday"] == "Saturday"][
    ["timestamp", "wm_yr_wk"]
]

map_wm_yr_wk_to_date = {}
for _, row in wm_yr_wk_to_date.iterrows():
    map_wm_yr_wk_to_date[row["wm_yr_wk"]] = row["timestamp"]

cleaned_sell_prices = raw_sell_prices.copy()
cleaned_sell_prices["timestamp"] = cleaned_sell_prices["wm_yr_wk"].apply(
    lambda x: map_wm_yr_wk_to_date[x]
)

del cleaned_sell_prices["wm_yr_wk"]

After:


In [ ]:
cleaned_sell_prices.head()

Same as before, we can convert the calendar data into a Temporian EventSet.


In [ ]:
sell_prices_evset = tp.from_pandas(
    cleaned_sell_prices, indexes=["store_id", "item_id"]
)

sell_prices_evset

Now that our data is in the Temporian format, we can delete the Pandas DataFrames to recover some memory.


In [ ]:
del raw_sales
del raw_sell_prices
del raw_calendar

## Feature engineering

The EventSets `sales_evset`, `calendar_evset` and `sell_prices_evset` are indexed by product.
To illustrate each of the newly computed features, we will plot them on a single product.


In [ ]:
# Select a product.
selected_index = sales_evset.get_arbitrary_index_key()

print("Selected product (selected_index):", selected_index)

The following arguments will be used for `tp.plot()`.


In [ ]:
plot_options = {
    # Only plot the selected product.
    "indexes": selected_index,
    # Make the plot interactive.
    "interactive": True,
    # Make sure the toolbar is visible
    "width_px": 500,
    # Only plot the data for the year 2015.
    "min_time": datetime(2015, 1, 1),
    "max_time": datetime(2015, 12, 31),
}

`all_features` will contain all of the generated features, and we'll use the prefix `f_` for all their names.


In [ ]:
all_features = [
    # The raw sales is the first feature.
    sales_evset["sales"].prefix("f_"),
]

### Label

The labels at time `t` are the sales at time `t+1..28`. This is computed with the `EventSet.leak()` operator.


In [ ]:
label_list: List[tp.EventSet] = []

# Prediction horizon. In the M5 competition, participants should forecast the next 28 days.
# You can predict less days to make the training section of this notebook run faster.
horizons = list(range(1, 28))

# For each of the horizons.
for horizon in horizons:
    # Leak the sales in the past.
    x = sales_evset["sales"].leak(tp.duration.days(horizon))

    # Resample the label to the sales data.
    x = x.resample(sales_evset)

    # Give a name to the label for book-keeping.
    x = x.rename(f"label_horizon_{horizon}_days")
    label_list.append(x)

labels = tp.glue(*label_list)

Let's see what the labels look like. Can you see how they shift more and more to the left while the horizon increases?


In [ ]:
labels.plot(**plot_options, max_num_plots=4)

### Past observations i.e. lagged sales

The model will have access to the daily sales in the last 3 days, as well as in the same day of the last and second-to-last weeks.

This is similar to computing the labels, except that the "shift" is in the opposite direction.


In [ ]:
lagged_sales_list: List[tp.EventSet] = []

for horizon in [1, 2, 3, 7, 14]:
    x = sales_evset["sales"].lag(tp.duration.days(horizon))
    x = x.resample(sales_evset)
    x = x.rename(f"f_sales_lag_{horizon}_d")
    lagged_sales_list.append(x)

feature_lagged_sales = tp.glue(*lagged_sales_list)

all_features.append(feature_lagged_sales)

In [ ]:
feature_lagged_sales.plot(**plot_options, max_num_plots=4)

### Moving statistics

Moving statistics can help the model identify global trends in the data (moving average) or periods of high or low volatility (moving standard deviation).


In [ ]:
moving_stats_list: List[tp.EventSet] = []

float_sales = sales_evset["sales"].cast(tp.float32)
for win_day in [7, 14, 28, 84]:
    win = tp.duration.days(win_day)

    x = float_sales.simple_moving_average(win).prefix(
        f"f_sma_{win_day}_"
    )
    moving_stats_list.append(x)

    x = float_sales.moving_standard_deviation(win).prefix(
        f"f_sd_{win_day}_"
    )
    moving_stats_list.append(x)

    x = float_sales.moving_sum(win).prefix(f"f_sum_{win_day}_")
    moving_stats_list.append(x)

feature_moving_stats = tp.glue(*moving_stats_list)

all_features.append(feature_moving_stats)

In [ ]:
feature_moving_stats.plot(**plot_options, max_num_plots=5)

### Calendar features

The model will not be able to use dates directly. Instead, we transform the date into numerical features that can be consumed by a model.


In [ ]:
calendar_list: List[tp.EventSet] = []
calendar_list.append(sales_evset.calendar_day_of_month())
calendar_list.append(sales_evset.calendar_day_of_week())
calendar_list.append(sales_evset.calendar_month())
feature_calendar = tp.glue(*calendar_list).prefix("f_")

all_features.append(feature_calendar)

In [ ]:
feature_calendar.plot(**plot_options, max_num_plots=5)

### Sales aggregated per department

When predicting the sales of a product, the sales of the other products in the same department can help forecasting.

Let's first group all the sales per department in each store, by removing the `item_id` feature from our EventSet's index.

In [ ]:
# `sales_per_dept` contains all the sales indexed by department / category / store / state.
# Aggregating the sales at other levels (e.g., by store) could also be useful.
sales_per_dept = sales_evset.drop_index("item_id", keep=False)

`sales_per_dept` still contains the sales of each item: each day, `sales_per_dept` contains one record for each item.
We need to aggregate those sales together on each day.

We compute the 28 days moving sum of sales for this department. To improve the quality of the final model, we could additionally use other time windows e.g. 1, 3, 7, 28.


In [ ]:
# Create a daily sampling.
sampling_once_a_day = sales_per_dept.unique_timestamps()

# Cumulative 28 days sum of sales, per department.
sum28_sales_per_dept = sales_per_dept["sales"].moving_sum(
        window_length=tp.duration.days(28),
        sampling=sampling_once_a_day
    )

# Give it a name for book-keeping.
sum28_sales_per_dept = sum28_sales_per_dept.prefix("f_sum28_per_dep_")

In [ ]:
sum28_sales_per_dept.plot(max_num_plots=3)

The sales of each department is then _propagated_ back to each corresponding product.

Note that two products from the same department will receive the same features.


In [ ]:
sum28_sales_per_dept = sum28_sales_per_dept.propagate(sales_evset, resample=True)

all_features.append(sum28_sales_per_dept)

In [ ]:
sum28_sales_per_dept.plot(max_num_plots=3)

### Special events

Special events such as holidays and sale periods can impact sales.


In [ ]:
# Propagate the global special event to each product.
special_event_per_product = calendar_evset.propagate(sales_evset, resample=True)

# Since special events are known in advance, we can leak future special events in the past.
#
# For example, on 22/12/2016, the model can know that it will be christmas in 2 days.
special_events_list: List[tp.EventSet] = []
special_events_list.append(special_event_per_product.prefix("f_"))

for leak in [5]:
    x = special_event_per_product.leak(tp.duration.days(leak))
    x = x.resample(sales_evset)
    x = x.prefix(f"f_leak{leak}_")
    special_events_list.append(x)

# To make the model more powerful, we could also apply some moving statistics such
# as moving-maximum to create features such as: is it christmas in the following N days.

feature_special_events = tp.glue(*special_events_list)

all_features.append(feature_special_events)

In [ ]:
feature_special_events.plot(**plot_options, max_num_plots=3)

### Review all the features

`all_features` contains all the features we want to feed to our model.
`tabular` is the data that will be used to train the model.


In [ ]:
tabular = tp.glue(*all_features, sales_evset["day"], labels)

tabular.schema

## Generate training and testing datasets

Compute all the new features and labels on the dataset.

The test dataset is the single day 1914. In this day, the model makes 28 predictions, one for each horizon, for each of the products.

For training, we use the data prior to the test day. We also remove the first 30 days as those days don't have enough data to compute good statistics.


In [ ]:
tabular_test = tabular.filter(tp.equal_scalar(tabular["day"], 1914))
tabular_train = tabular.filter(
    (tabular["day"] >= 30) & (tabular["day"] < 1914)
)

Let's compute the test dataset.


In [ ]:
test_dataset = tp.to_pandas(tabular_test)

test_dataset.head()

Let's compute the train dataset.


In [ ]:
train_dataset = tp.to_pandas(tabular_train)

train_dataset.head()

We can convert those EventSets to Pandas DataFrames and save them to csv files for later use.


In [ ]:
test_dataset.to_csv(os.path.join(work_directory, "test.csv"), index=False)
train_dataset.to_csv(os.path.join(work_directory, "train.csv"), index=False)

In [ ]:
!head tmp/temporian_m5/train.csv -n 5

### Training and evaluating a simple model

Now that we have pre-processed data, we can train a model. We will train a collection of Gradient Boosted Trees using TensorFlow Decision Forests.


In [ ]:
%pip install tensorflow tensorflow_decision_forests -q

import tensorflow as tf
import tensorflow_decision_forests as tfdf

In [ ]:
# The name of the labels and features.
label_names = [x for x in test_dataset.columns if x.startswith("label_")]
feature_names = [x for x in test_dataset.columns if x.startswith("f_")] + [
    # The model also has access to the meta-data.
    "dept_id",
    "cat_id",
    "store_id",
    "state_id",
] 

print("label_names:", label_names)
print("feature_names:", feature_names)

The Pandas DataFrame can be converted into a TensorFlow dataset.


In [ ]:
def dataset_pandas_to_tensorflow(df):
    features = {k: df[k] for k in feature_names}
    labels = {k: df[k] for k in label_names}
    return tf.data.Dataset.from_tensor_slices((features, labels)).batch(100)


tf_test_dataset = dataset_pandas_to_tensorflow(test_dataset)
tf_train_dataset = dataset_pandas_to_tensorflow(train_dataset)

We can train our model.


In [ ]:
model_path = os.path.join(work_directory, "gbm.model")
if os.path.exists(model_path):
    model = tf.keras.models.load_model(model_path)
else:
    model = tfdf.keras.GradientBoostedTreesModel(
        multitask=[
            tfdf.keras.MultiTaskItem(x, task=tfdf.keras.Task.REGRESSION)
            for x in label_names
        ],
        num_trees=50,  # Increase the number of trees (e.g., 100) for a better model.
        verbose=2, # Remove to make the cell less verbose.
        num_threads=8,
    )
    
    model.fit(tf_train_dataset)
    model.save(model_path)


Let's generate the predictions on the test dataset.


In [ ]:
tf_predictions = model.predict(tf_test_dataset, verbose=0)

Let's convert those TensorFlow predictions into a Temporian EventSet. This way we will be able to plot the predictions alongside the real sales.


In [ ]:
index_names = tabular_test.schema.index_names()

raw_predicted_sales = defaultdict(list)
timestamps = []

# For each product
for row_idx in range(len(test_dataset)):
    original_timestamp = datetime.fromtimestamp(
        test_dataset["timestamp"][row_idx], tz=timezone.utc
    )

    # For each prediction horizon
    for horizon, predictions in zip(horizons, tf_predictions.values()):
        timestamps.append(original_timestamp + timedelta(days=horizon))

        for k in index_names:
            raw_predicted_sales[k].append(test_dataset[k][row_idx])

        raw_predicted_sales["prediction_sales"].append(predictions[row_idx, 0])

# Convert the dictionary of values into an event set.
prediction_evset = tp.event_set(
    timestamps, raw_predicted_sales, indexes=index_names
)

Finally, we can plot the real sales and predicted sales.

**Remember:** Update the variable `horizons` to change the number of days the model is predicting (14 by default).

### Plot predictions

Let's now show some predictions and compare them against the real sales. The sales before the test day 1914 are considered past sales, and then we compare the predictions against real sales after the test day.

In this cell we'll just use the standard plots, and then we'll customize them to better compare predictions against ground-truth.

In [ ]:
real_sales_evset = sales_evset["sales"].filter(sales_evset["day"] >= 1914)

past_sales_evset = sales_evset["sales"].filter((sales_evset["day"] <= 1914) & (sales_evset["day"] >= 1914 - 40))

# Rename and plot together
real_sales_evset = real_sales_evset.rename("real_sales")
past_sales_evset = past_sales_evset.rename("past_sales")
prediction_evset = prediction_evset.rename("predicted_sales")

tp.plot(
    [real_sales_evset, prediction_evset, past_sales_evset],
    max_num_plots=3*2,
    merge=True,
)


How does that model work? We can look at the variable importance to get an idea of what feature matters the most with `model.summary()`.

Note that this model contains in fact one individual model for each horizon. In other words, this model contains 28 sub models!